# Normalização dos Corpora

In [1]:
import os
os.chdir('..')

In [2]:
from gensim.models import Word2Vec, Doc2Vec
from gensim.models.doc2vec import TaggedDocument

In [3]:
from nlputils.lexical.normalizer import Normalizer

In [4]:
normalizer = Normalizer()

In [5]:
def load_sentences(corpus_dir):   
    all_sentences = []
    for file_path in os.listdir(corpus_dir):
        with open(corpus_dir + file_path) as fp:
            for line in fp:
                line = normalizer.to_lowercase(line)
                sentences = normalizer.tokenize_sentences(line)
                sentences = [normalizer.tokenize_words(sent) for sent in sentences]
                all_sentences.extend(sentences)
    return all_sentences

## Córpus Telefonia

In [8]:
telefonia_sentences = load_sentences('data/corpora/telefonia/')

In [9]:
w2vmodel_telefonia = Word2Vec(telefonia_sentences, size=200, window=5, min_count=3, workers=4)

In [30]:
w2vmodel_telefonia.wv.most_similar('telefone')

[('serviço', 0.9998871088027954),
 ('trump', 0.9998842477798462),
 ('hoje', 0.999881386756897),
 ('brasileiro', 0.9998754262924194),
 ('outros', 0.999870240688324),
 ('depois', 0.9998632073402405),
 ('nunca', 0.9998611211776733),
 ('baixo', 0.9998608827590942),
 ('nesse', 0.9998584985733032),
 ('premium', 0.9998577833175659)]

## Córpus Saúde

In [31]:
saude_sentences = load_sentences('data/corpora/saude/')

In [32]:
w2vmodel_saude = Word2Vec(saude_sentences, size=200, window=5, min_count=3, workers=4)

In [33]:
w2vmodel_saude.wv.most_similar('doença')

[('faz', 0.9993070960044861),
 ('vírus', 0.9989165663719177),
 ('feito', 0.998900830745697),
 ('impacto', 0.9987791180610657),
 ('grande', 0.998769998550415),
 ('boa', 0.9987643361091614),
 ('remédio', 0.998640239238739),
 ('homem', 0.9986361861228943),
 ('medicamento', 0.9986277222633362),
 ('sair', 0.9985885620117188)]